In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import cv2
import random


In [2]:
# Parameters of data
img_size = 28
img_flat_size = img_size * img_size

num_label = 10 # 0 ~ 9

# Parameters of training
Learning_rate = 0.0005
epsilon = 1e-2

num_iter = 5100
batch_size = 100

validation_ratio = 0.1
gpu_fraction = 0.5

# Ensemble networks
networks = ['network1', 'network2', 'network3', 'network4', 'network5']

#keep_prob = tf.placeholder(tf.float32)
#batch_prob = tf.placeholder(tf.bool)

In [3]:
# Import MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', validation_size=0)

# Dataset for train, test, validation
test_len = mnist.test.images.shape[0]
validation_len = int(test_len * validation_ratio)

train_x = mnist.train.images
test_x = mnist.test.images[validation_len : test_len, :]
validation_x = mnist.test.images[ : validation_len, :]

train_y_index = mnist.train.labels
test_y_index = mnist.test.labels[validation_len : test_len]
validation_y_index = mnist.test.labels[ : validation_len]

train_y = np.zeros([train_y_index.shape[0], num_label])
test_y = np.zeros([test_y_index.shape[0], num_label])
validation_y = np.zeros([validation_y_index.shape[0], num_label])

for i in range(train_y.shape[0]):
    train_y[i, train_y_index[i]] = 1

for i in range(test_y.shape[0]):
    test_y[i, test_y_index[i]] = 1
    
for i in range(validation_y.shape[0]):
    validation_y[i, validation_y_index[i]] = 1
    
print("\nTraining X shape: " + str(train_x.shape))
print("Testing X shape: " + str(test_x.shape))
print("Validation X shape: " + str(validation_x.shape))

print("\nTraining Y shape: " + str(train_y.shape))
print("Testing Y shape: " + str(test_y.shape))
print("Validation Y shape: " + str(validation_y.shape))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

Training X shape: (60000, 784)
Testing X shape: (9000, 784)
Validation X shape: (1000, 784)

Training Y shape: (60000, 10)
Testing Y shape: (9000, 10)
Validation Y shape: (1000, 10)


In [4]:
folders_notMNIST = os.listdir('C:/Users/Alchera/Desktop/jaeseung/code/uncertainty_project/notMNIST_small')

NotMNIST_x_list = []
NotMNIST_y_list = []

for idx, folder in enumerate(folders_notMNIST):
    files_notMNIST = os.listdir('./notMNIST_small/' + folder)
    
    for file in files_notMNIST:
        img_NotMNIST = cv2.imread('./notMNIST_small/' + folder + '/' + file, 0)
        NotMNIST_flat = np.reshape(img_NotMNIST, (img_flat_size))
        NotMNIST_x_list.append(NotMNIST_flat)
        
        label_temp = np.zeros([num_label])
        label_temp[idx] = 1
        
        NotMNIST_y_list.append(label_temp)
        
NotMNIST_x = np.stack(NotMNIST_x_list, axis = 0)
NotMNIST_y = np.stack(NotMNIST_y_list, axis = 0)

print("NotMNIST X shape: " + str(NotMNIST_x.shape))
print("NotMNIST Y shape: " + str(NotMNIST_y.shape))

NotMNIST X shape: (18674, 784)
NotMNIST Y shape: (18674, 10)


In [5]:
tf.reset_default_graph()

keep_prob = tf.placeholder(tf.float32)
batch_prob = tf.placeholder(tf.bool)
    
# Get Variables
def weight_variable(name, shape):
    return tf.get_variable(name, shape = shape, initializer = tf.contrib.layers.xavier_initializer())

def bias_variable(name, shape):
    return tf.get_variable(name, shape = shape, initializer = tf.contrib.layers.xavier_initializer())

# Get networks
def get_network(network_name):
    
    x_image = tf.placeholder(tf.float32, shape = [None, img_flat_size*1])
    y_label = tf.placeholder(tf.float32, shape = [None, num_label])
        
    n_hidden_1 = 200 # 1st layer number of features
    n_hidden_2 = 200 # 2nd layer number of features
    n_hidden_3 = 200 # 3rd layer
    
    with tf.variable_scope(network_name):
        
        # Densely connect layer variables
        w1 = weight_variable(network_name + '_w1', [img_flat_size, n_hidden_1])
        w2 = weight_variable(network_name + '_w2', [n_hidden_1,n_hidden_2])
        w3 = weight_variable(network_name + '_w3', [n_hidden_2,n_hidden_3])
        w4 = weight_variable(network_name + '_w4', [n_hidden_3,num_label])
        
        b1 = bias_variable(network_name + '_b1', [n_hidden_1])
        b2 = bias_variable(network_name + '_b2', [n_hidden_2])
        b3 = bias_variable(network_name + '_b3', [n_hidden_3])
        b4 = bias_variable(network_name + '_b4', [num_label])
        
        
    # Network
    L1 = tf.nn.relu(tf.matmul(x_image,w1) + b1)
    #L1 = tf.contrib.layers.batch_norm(L1, center=True, scale=False, is_training=batch_prob)
    L1 = tf.nn.dropout(L1, keep_prob = keep_prob)
    
    L2 = tf.nn.relu(tf.matmul(L1,w2) + b2)
    #L2 = tf.contrib.layers.batch_norm(L2, center=True, scale=False, is_training=batch_prob)
    L2 = tf.nn.dropout(L2, keep_prob = keep_prob)
    
    L3 = tf.nn.relu(tf.matmul(L2,w3) + b3)
    #L3 = tf.contrib.layers.batch_norm(L3, center=True, scale=False, is_training=batch_prob)
    L3 = tf.nn.dropout(L3, keep_prob = keep_prob)
    
    logits = tf.matmul(L3,w4) + b4
    output = tf.nn.softmax(logits)
    
    # Brier Score 
    loss = tf.reduce_mean(tf.div(tf.reduce_sum(tf.square(tf.subtract(output, y_label)), axis = 1), num_label), axis = 0) 
   
    # Get trainable variables
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, network_name)
    train_opt = tf.train.AdamOptimizer(Learning_rate).minimize(loss, var_list = train_vars)
    
    return x_image, y_label, output, loss, train_opt, train_vars


# Make batch data 
def making_batch(data_size, sample_size, data_x, data_y):
    # Making batches(testing)
    batch_idx = np.random.choice(data_size, sample_size)
    
    batch_x = np.zeros([sample_size, img_flat_size])
    batch_y = np.zeros([sample_size, num_label])
        
    for i in range(batch_idx.shape[0]):
        batch_x[i,:] = np.reshape(data_x[batch_idx[i], :], (img_flat_size))
        batch_y[i,:] = data_y[batch_idx[i], :] 
        
    return batch_x, batch_y

def making_batch_formlp(data_size, sample_size, data_x, data_y):
    # Making batches(testing)
    batch_idx = np.random.choice(data_size, sample_size)
    
    batch_x = np.zeros([sample_size, img_flat_size])
    batch_y = np.zeros([sample_size, num_label])
        
    for i in range(batch_idx.shape[0]):
        batch_x[i,:] = data_x[batch_idx[i], :]
        batch_y[i,:] = data_y[batch_idx[i], :]
        
    return batch_x, batch_y

# Calculate Accuracy
def get_accuracy(prediction, label):
    count_correct = 0
    for j in range(prediction.shape[0]):
        if np.argmax(label[j, :]) == np.argmax(prediction[j, :]):
            count_correct += 1.0
    
    acc = count_correct / label.shape[0]
    return acc

In [6]:
# Ensemble Networks
x_list = []
y_list = []
output_list = []
loss_list = []
train_list = []
train_var_list = []

# Train each ensemble network
for i in range(len(networks)):
    x_image, y_label, output, loss, train_opt, train_vars = get_network(networks[i])

    x_list.append(x_image)
    y_list.append(y_label)
    output_list.append(output)
    loss_list.append(loss)
    train_list.append(train_opt)
    train_var_list.append(train_vars)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.


In [7]:
# Create Session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpu_fraction

sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [8]:
# Initialize data for printing
loss_check     = np.zeros(len(networks))
acc_check      = np.zeros(len(networks))
acc_check_val = np.zeros(len(networks))
acc_check_val_final = 0

# Set parameters for printing and validating
num_print = 100
test_size = 10

train_data_num = train_x.shape[0]
val_data_num  = validation_x.shape[0]

for iter in range(num_iter):
    output_temp = np.zeros([test_size, num_label])

    # Making batches(validation)
    batch_x_val, batch_y_val = making_batch_formlp(val_data_num, test_size, validation_x, validation_y)
        
    for i in range(len(networks)):
        # Making batches(training)
        batch_x, batch_y = making_batch_formlp(train_data_num, batch_size, train_x, train_y)
       
        # Training
        _, loss, prob = sess.run([train_list[i], loss_list[i], output_list[i]], 
                                 feed_dict = {x_list[i]: batch_x, y_list[i]: batch_y,keep_prob:0.9})
        
        # Validation
        loss_test, prob_test = sess.run([loss_list[i], output_list[i]], 
                                         feed_dict = {x_list[i]: batch_x_val, y_list[i]: batch_y_val,keep_prob:0.9})
        
        # Add test prediction for get final prediction
        output_temp += prob_test
        # Calculate Accuracy (Training)
        acc_training = get_accuracy(prob, batch_y)
 
        # Calculate Accuracy (validating)
        acc_validating = get_accuracy(prob_test, batch_y_val)
                
        # Get accuracy and loss for each network
        acc_check[i] += acc_training
        acc_check_val[i] += acc_validating
        loss_check[i] += loss
    
    # Get final test prediction
    prob_val_final = output_temp / len(networks)
    # Calculate Accuracy (Validating final)
    acc_validating_final = get_accuracy(prob_val_final, batch_y_val)
    acc_check_val_final += acc_validating_final
    
    if iter % num_print == 0 and iter != 0:
        print(('-------------------------') + ' Iteration: ' + str(iter) + ' -------------------------')
        print('Average Loss(Brier score): ' + str(loss_check / num_print))
        print('Training Accuracy: ' + str(acc_check / num_print))
        print('Validating Accuracy: ' + str(acc_check_val / num_print))
        print('Final Validating Accuracy: ' + str(acc_check_val_final / num_print))
        print('\n')
        
        loss_check = np.zeros(len(networks))
        acc_check = np.zeros(len(networks))
        acc_check_val = np.zeros(len(networks))
        acc_check_val_final = 0

------------------------- Iteration: 100 -------------------------
Average Loss(Brier score): [0.0355497  0.03553126 0.03514758 0.03547973 0.03468377]
Training Accuracy: [0.7721 0.7655 0.7756 0.7663 0.7746]
Validating Accuracy: [0.774 0.765 0.764 0.747 0.768]
Final Validating Accuracy: 0.7949999999999997


------------------------- Iteration: 200 -------------------------
Average Loss(Brier score): [0.01273894 0.01274223 0.01311696 0.01320876 0.01311437]
Training Accuracy: [0.9174 0.9185 0.9138 0.9142 0.9149]
Validating Accuracy: [0.914 0.904 0.919 0.906 0.91 ]
Final Validating Accuracy: 0.9220000000000002


------------------------- Iteration: 300 -------------------------
Average Loss(Brier score): [0.01024606 0.01121432 0.01114086 0.01042073 0.01125353]
Training Accuracy: [0.9332 0.9264 0.928  0.9332 0.9263]
Validating Accuracy: [0.925 0.923 0.931 0.929 0.923]
Final Validating Accuracy: 0.9410000000000003


------------------------- Iteration: 400 -------------------------
Average L

------------------------- Iteration: 2800 -------------------------
Average Loss(Brier score): [0.00287178 0.00284839 0.00247213 0.00247555 0.0027242 ]
Training Accuracy: [0.9822 0.9826 0.9849 0.9841 0.9832]
Validating Accuracy: [0.973 0.969 0.959 0.976 0.966]
Final Validating Accuracy: 0.9740000000000001


------------------------- Iteration: 2900 -------------------------
Average Loss(Brier score): [0.00204939 0.00218836 0.00236773 0.00281082 0.00248009]
Training Accuracy: [0.9878 0.9866 0.9852 0.9824 0.9846]
Validating Accuracy: [0.966 0.969 0.969 0.972 0.969]
Final Validating Accuracy: 0.9740000000000002


------------------------- Iteration: 3000 -------------------------
Average Loss(Brier score): [0.00263732 0.00252904 0.00254156 0.00263208 0.0023003 ]
Training Accuracy: [0.9843 0.9855 0.9845 0.9838 0.9862]
Validating Accuracy: [0.971 0.974 0.963 0.976 0.974]
Final Validating Accuracy: 0.981


------------------------- Iteration: 3100 -------------------------
Average Loss(Brier

In [18]:
#testset 중 9000개의 인덱스 추출
MNIST_sample_index = np.random.choice(test_x.shape[0],9000)
print(MNIST_sample_index)
Macc_final_final_en = 0
Macc_final_final_on = 0

output_sample     = []
output_sample_one = []

m_en = []
m_one = []

for i, index in enumerate(MNIST_sample_index):
    img_MNIST = np.reshape(test_x[index], (img_flat_size))
    output_test = np.zeros([1, num_label])

    #각 네트위크별로 prob를 구한다음 계속 더함
    for net_index in range(len(networks)):
        x_temp = np.reshape(test_x[index, :], (1, img_flat_size))
        y_temp = np.reshape(test_y[index, :], (1, num_label))

        loss_temp, prob_temp = sess.run([loss_list[net_index], output_list[net_index]], 
                                         feed_dict = {x_list[net_index]: x_temp, y_list[net_index]: y_temp,keep_prob:0.9})        
        # Add test prediction for get final prediction
        output_test += prob_temp
 
    # Get final test prediction
    prob_temp_final = output_test / len(networks)
    prob_temp_one   = prob_temp
    
    # Get accuracy
    acc_final_check_en_m = get_accuracy(prob_temp_final, y_temp)
    Macc_final_final_en += acc_final_check_en_m
    acc_final_check_on_m = get_accuracy(prob_temp_one, y_temp)
    Macc_final_final_on += acc_final_check_on_m
    
    m_en.append(acc_final_check_en_m)
    m_one.append(acc_final_check_on_m)
    output_sample.append(prob_temp_final)
    output_sample_one.append(prob_temp_one)

[3454 3413 8499 ... 4008 7998  932]


In [26]:
print("====================== Ensemble Result ======================")
array_ensemble_MNIST = np.zeros([9000])
conf = 0.2
count_ex_m_en = 0
correction_m_en = 0
for i in range(len(output_sample)):
    
    idx_sample = np.argmax(output_sample[i])
    max_prob = output_sample[i][0, idx_sample]
    array_ensemble_MNIST[i] = max_prob
    
    if max_prob >= conf:
        
        count_ex_m_en += 1
        correction_m_en += m_en[i]
        
    print(str(i+1) + 'th sample: label = ' + str(idx_sample) + ', Confidence = ' + str(max_prob) + ', Correction = ' + str(m_en[i]))
print("Accuracy: " + str(Macc_final_final_en/9000) + " Condition: " + str(correction_m_en))
print("Total: " + str(count_ex_m_en))

====================== Ensemble Result ======================
Accuracy: 0.9816666666666667 Condition: 8835.0
Total: 9000


In [11]:
print("\n====================== SingleNet Result ======================")
array_single_MNIST = np.zeros([9000])
count_ex_m_on = 0
correction_m_on = 0
for i in range(len(output_sample_one)):
    
    idx_sample = np.argmax(output_sample_one[i])
    max_prob = output_sample_one[i][0, idx_sample]
    array_single_MNIST[i] = max_prob
    
    if max_prob >= conf:
        count_ex_m_on += 1
        correction_m_on += m_one[i]
        
    print(str(i+1) + 'th sample: label = ' + str(idx_sample) + ', Confidence = ' + str(max_prob) + ', Correction = ' + str(m_one[i]))
print("Accuracy: " + str(Macc_final_final_on/9000) + " Condition: " + str(correction_m_on))
print("Total: " +str(count_ex_m_on))


====================== SingleNet Result ======================
1th sample: label = 2, Confidence = 0.99999523, Correction = 1.0
2th sample: label = 7, Confidence = 0.99973243, Correction = 1.0
3th sample: label = 7, Confidence = 0.99999833, Correction = 1.0
4th sample: label = 1, Confidence = 0.9986337, Correction = 1.0
5th sample: label = 2, Confidence = 0.99999416, Correction = 1.0
6th sample: label = 9, Confidence = 0.99990964, Correction = 1.0
7th sample: label = 5, Confidence = 0.99982315, Correction = 1.0
8th sample: label = 5, Confidence = 0.9981482, Correction = 1.0
9th sample: label = 8, Confidence = 0.9945807, Correction = 1.0
10th sample: label = 9, Confidence = 0.9999095, Correction = 1.0
11th sample: label = 8, Confidence = 0.9353198, Correction = 0.0
12th sample: label = 2, Confidence = 0.9999857, Correction = 1.0
13th sample: label = 0, Confidence = 0.9977308, Correction = 1.0
14th sample: label = 7, Confidence = 0.9999689, Correction = 1.0
15th sample: label = 1, Confi

2570th sample: label = 4, Confidence = 0.9998865, Correction = 1.0
2571th sample: label = 0, Confidence = 0.99999285, Correction = 1.0
2572th sample: label = 5, Confidence = 0.99989176, Correction = 1.0
2573th sample: label = 1, Confidence = 0.9997968, Correction = 1.0
2574th sample: label = 6, Confidence = 0.7699767, Correction = 1.0
2575th sample: label = 4, Confidence = 1.0, Correction = 1.0
2576th sample: label = 1, Confidence = 0.9996532, Correction = 1.0
2577th sample: label = 1, Confidence = 0.85471535, Correction = 1.0
2578th sample: label = 4, Confidence = 0.999982, Correction = 1.0
2579th sample: label = 7, Confidence = 0.99989414, Correction = 1.0
2580th sample: label = 3, Confidence = 0.99999595, Correction = 1.0
2581th sample: label = 4, Confidence = 0.9999994, Correction = 1.0
2582th sample: label = 7, Confidence = 0.99998975, Correction = 1.0
2583th sample: label = 2, Confidence = 0.9992229, Correction = 1.0
2584th sample: label = 5, Confidence = 0.99980956, Correction =

4890th sample: label = 5, Confidence = 0.999982, Correction = 1.0
4891th sample: label = 5, Confidence = 0.9999949, Correction = 1.0
4892th sample: label = 0, Confidence = 0.99979395, Correction = 1.0
4893th sample: label = 8, Confidence = 0.9999988, Correction = 1.0
4894th sample: label = 9, Confidence = 0.9999919, Correction = 1.0
4895th sample: label = 7, Confidence = 0.9980987, Correction = 1.0
4896th sample: label = 6, Confidence = 0.99995387, Correction = 1.0
4897th sample: label = 2, Confidence = 0.9994368, Correction = 1.0
4898th sample: label = 9, Confidence = 0.99909794, Correction = 1.0
4899th sample: label = 6, Confidence = 0.99993193, Correction = 1.0
4900th sample: label = 7, Confidence = 0.9999709, Correction = 1.0
4901th sample: label = 1, Confidence = 0.995884, Correction = 1.0
4902th sample: label = 2, Confidence = 0.9998698, Correction = 1.0
4903th sample: label = 5, Confidence = 0.99990976, Correction = 1.0
4904th sample: label = 1, Confidence = 0.9998547, Correctio

7069th sample: label = 4, Confidence = 0.9999913, Correction = 1.0
7070th sample: label = 8, Confidence = 0.99869955, Correction = 1.0
7071th sample: label = 8, Confidence = 0.999979, Correction = 1.0
7072th sample: label = 3, Confidence = 0.99999964, Correction = 1.0
7073th sample: label = 3, Confidence = 0.99994695, Correction = 1.0
7074th sample: label = 6, Confidence = 0.9999951, Correction = 1.0
7075th sample: label = 9, Confidence = 0.99980253, Correction = 1.0
7076th sample: label = 6, Confidence = 0.99999535, Correction = 1.0
7077th sample: label = 4, Confidence = 0.9907155, Correction = 1.0
7078th sample: label = 0, Confidence = 0.99999976, Correction = 1.0
7079th sample: label = 4, Confidence = 0.9995382, Correction = 1.0
7080th sample: label = 9, Confidence = 0.99999964, Correction = 1.0
7081th sample: label = 1, Confidence = 0.99971455, Correction = 1.0
7082th sample: label = 6, Confidence = 0.99868804, Correction = 1.0
7083th sample: label = 0, Confidence = 0.99981254, Cor

In [29]:
NotMNIST_sample_index = np.random.choice(NotMNIST_x.shape[0], 9000)

Nacc_final_final_en = 0
Nacc_final_final_on = 0

output_sample     = []
output_sample_one = []

n_en = []
n_one = []

for i, index in enumerate(NotMNIST_sample_index):    
    output_test = np.zeros([1, num_label])
    
    for net_index in range(len(networks)):
        x_temp = np.reshape(NotMNIST_x[index, :], (1, img_flat_size))
        y_temp = np.reshape(NotMNIST_y[index, :], (1, num_label))
        
        loss_temp, prob_temp = sess.run([loss_list[net_index], output_list[net_index]], 
                                         feed_dict = {x_list[net_index]: x_temp, y_list[net_index]: y_temp,keep_prob:0.9})
        
        # Add test prediction for get final prediction
        output_test += prob_temp
        
    # Get final test prediction
    prob_temp_final = output_test / len(networks)
    prob_temp_one   = prob_temp
    
     # Get accuracy
    acc_final_check_en_n = 0
    Nacc_final_final_en += acc_final_check_en_n
    acc_final_check_on_n = 0
    Nacc_final_final_on += acc_final_check_on_n

    n_en.append(acc_final_check_en_n)
    n_one.append(acc_final_check_on_n)
    
    output_sample.append(prob_temp_final)
    output_sample_one.append(prob_temp_one)

In [37]:
print("====================== Ensemble Result ======================")
array_ensemble_NotMNIST = np.zeros([9000])
count_ex_n_en = 0
tot_prob_en = 0
conf = 0.2
for i in range(len(output_sample)):   
    idx_sample = np.argmax(output_sample[i])
    max_prob = output_sample[i][0, idx_sample]
    array_ensemble_NotMNIST[i] = max_prob
    tot_prob_en += max_prob
    
    if max_prob >= conf:
        count_ex_n_en += 1
    print(str(i+1) + 'th sample: label = ' + str(idx_sample) + ', Confidence = ' + str(max_prob))
            
print("Accuracy: " + str(Nacc_final_final_en/9000))
print('Avg confidence: ' + str(tot_prob_en/9000))
print('Satisfying Ensemble NotMNIST: ' + str(count_ex_n_en))
total_count_en = count_ex_m_en + count_ex_n_en
acc_m_en = (correction_m_en/total_count_en)*100
print('Total accuracy: ' + str(acc_m_en))

====================== Ensemble Result ======================
Accuracy: 0.0
Avg confidence: 0.6852795740368225
Satisfying Ensemble NotMNIST: 9000
Total accuracy: 49.083333333333336


In [14]:
print("\n====================== SingleNet Result ======================")
array_single_NotMNIST = np.zeros([10800])
count_ex_n_one = 0
tot_prob_on = 0
for i in range(len(output_sample_one)):    
    idx_sample = np.argmax(output_sample_one[i])
    max_prob = output_sample_one[i][0, idx_sample]
    array_single_NotMNIST[i] = max_prob
    tot_prob_on += max_prob
    
    if max_prob >= conf:
        count_ex_n_one += 1
        
    print(str(i+1) + 'th sample: label = ' + str(idx_sample) + ', Confidence = ' + str(max_prob))
print("Accuracy: " + str(Nacc_final_final_on/9000))
print("Avg confidence : " + str(tot_prob_on/9000))
print('Satisfying Single NotMNIST: ' + str(count_ex_n_one))


====================== SingleNet Result ======================
1th sample: label = 7, Confidence = 1.0
2th sample: label = 2, Confidence = 1.0
3th sample: label = 3, Confidence = 1.0
4th sample: label = 3, Confidence = 1.0
5th sample: label = 5, Confidence = 1.0
6th sample: label = 5, Confidence = 1.0
7th sample: label = 5, Confidence = 1.0
8th sample: label = 2, Confidence = 1.0
9th sample: label = 7, Confidence = 1.0
10th sample: label = 6, Confidence = 1.0
11th sample: label = 2, Confidence = 1.0
12th sample: label = 8, Confidence = 1.0
13th sample: label = 2, Confidence = 1.0
14th sample: label = 4, Confidence = 1.0
15th sample: label = 9, Confidence = 1.0
16th sample: label = 9, Confidence = 1.0
17th sample: label = 9, Confidence = 1.0
18th sample: label = 7, Confidence = 1.0
19th sample: label = 7, Confidence = 1.0
20th sample: label = 2, Confidence = 1.0
21th sample: label = 3, Confidence = 1.0
22th sample: label = 7, Confidence = 1.0
23th sample: label = 8, Confidence = 1.0
24

2543th sample: label = 5, Confidence = 1.0
2544th sample: label = 6, Confidence = 1.0
2545th sample: label = 7, Confidence = 1.0
2546th sample: label = 7, Confidence = 1.0
2547th sample: label = 6, Confidence = 1.0
2548th sample: label = 7, Confidence = 1.0
2549th sample: label = 2, Confidence = 1.0
2550th sample: label = 7, Confidence = 1.0
2551th sample: label = 3, Confidence = 1.0
2552th sample: label = 6, Confidence = 1.0
2553th sample: label = 3, Confidence = 1.0
2554th sample: label = 3, Confidence = 1.0
2555th sample: label = 2, Confidence = 1.0
2556th sample: label = 7, Confidence = 1.0
2557th sample: label = 2, Confidence = 1.0
2558th sample: label = 5, Confidence = 1.0
2559th sample: label = 5, Confidence = 1.0
2560th sample: label = 6, Confidence = 1.0
2561th sample: label = 5, Confidence = 1.0
2562th sample: label = 9, Confidence = 1.0
2563th sample: label = 2, Confidence = 1.0
2564th sample: label = 6, Confidence = 1.0
2565th sample: label = 1, Confidence = 1.0
2566th samp

2805th sample: label = 5, Confidence = 1.0
2806th sample: label = 2, Confidence = 1.0
2807th sample: label = 7, Confidence = 1.0
2808th sample: label = 2, Confidence = 1.0
2809th sample: label = 3, Confidence = 1.0
2810th sample: label = 8, Confidence = 1.0
2811th sample: label = 5, Confidence = 1.0
2812th sample: label = 3, Confidence = 1.0
2813th sample: label = 6, Confidence = 1.0
2814th sample: label = 7, Confidence = 1.0
2815th sample: label = 7, Confidence = 1.0
2816th sample: label = 3, Confidence = 1.0
2817th sample: label = 2, Confidence = 1.0
2818th sample: label = 9, Confidence = 1.0
2819th sample: label = 7, Confidence = 1.0
2820th sample: label = 7, Confidence = 1.0
2821th sample: label = 7, Confidence = 1.0
2822th sample: label = 2, Confidence = 1.0
2823th sample: label = 6, Confidence = 1.0
2824th sample: label = 7, Confidence = 1.0
2825th sample: label = 8, Confidence = 1.0
2826th sample: label = 3, Confidence = 1.0
2827th sample: label = 2, Confidence = 1.0
2828th samp

5042th sample: label = 7, Confidence = 1.0
5043th sample: label = 6, Confidence = 1.0
5044th sample: label = 7, Confidence = 1.0
5045th sample: label = 7, Confidence = 1.0
5046th sample: label = 1, Confidence = 1.0
5047th sample: label = 7, Confidence = 1.0
5048th sample: label = 7, Confidence = 1.0
5049th sample: label = 7, Confidence = 1.0
5050th sample: label = 3, Confidence = 1.0
5051th sample: label = 5, Confidence = 1.0
5052th sample: label = 6, Confidence = 1.0
5053th sample: label = 3, Confidence = 1.0
5054th sample: label = 5, Confidence = 1.0
5055th sample: label = 2, Confidence = 1.0
5056th sample: label = 0, Confidence = 1.0
5057th sample: label = 3, Confidence = 1.0
5058th sample: label = 7, Confidence = 1.0
5059th sample: label = 0, Confidence = 1.0
5060th sample: label = 7, Confidence = 1.0
5061th sample: label = 2, Confidence = 1.0
5062th sample: label = 2, Confidence = 1.0
5063th sample: label = 5, Confidence = 1.0
5064th sample: label = 6, Confidence = 1.0
5065th samp

5313th sample: label = 8, Confidence = 1.0
5314th sample: label = 6, Confidence = 1.0
5315th sample: label = 3, Confidence = 1.0
5316th sample: label = 6, Confidence = 1.0
5317th sample: label = 8, Confidence = 1.0
5318th sample: label = 7, Confidence = 1.0
5319th sample: label = 7, Confidence = 1.0
5320th sample: label = 8, Confidence = 1.0
5321th sample: label = 5, Confidence = 1.0
5322th sample: label = 8, Confidence = 1.0
5323th sample: label = 5, Confidence = 1.0
5324th sample: label = 5, Confidence = 1.0
5325th sample: label = 7, Confidence = 1.0
5326th sample: label = 2, Confidence = 1.0
5327th sample: label = 8, Confidence = 1.0
5328th sample: label = 5, Confidence = 1.0
5329th sample: label = 5, Confidence = 1.0
5330th sample: label = 5, Confidence = 1.0
5331th sample: label = 7, Confidence = 1.0
5332th sample: label = 6, Confidence = 1.0
5333th sample: label = 7, Confidence = 1.0
5334th sample: label = 4, Confidence = 1.0
5335th sample: label = 5, Confidence = 1.0
5336th samp

7893th sample: label = 6, Confidence = 1.0
7894th sample: label = 3, Confidence = 1.0
7895th sample: label = 7, Confidence = 1.0
7896th sample: label = 2, Confidence = 1.0
7897th sample: label = 7, Confidence = 1.0
7898th sample: label = 7, Confidence = 1.0
7899th sample: label = 2, Confidence = 1.0
7900th sample: label = 5, Confidence = 1.0
7901th sample: label = 5, Confidence = 1.0
7902th sample: label = 3, Confidence = 1.0
7903th sample: label = 9, Confidence = 1.0
7904th sample: label = 5, Confidence = 1.0
7905th sample: label = 7, Confidence = 1.0
7906th sample: label = 7, Confidence = 0.9986313
7907th sample: label = 5, Confidence = 1.0
7908th sample: label = 5, Confidence = 1.0
7909th sample: label = 7, Confidence = 1.0
7910th sample: label = 2, Confidence = 1.0
7911th sample: label = 8, Confidence = 1.0
7912th sample: label = 5, Confidence = 1.0
7913th sample: label = 3, Confidence = 1.0
7914th sample: label = 3, Confidence = 1.0
7915th sample: label = 2, Confidence = 1.0
7916t